In [ ]:
from gen_finetune.run_finetune_experiment import get_dataset, prep_dataset
from pathlib import Path
import json
import transformers
from transformers import pipeline
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version used by PyTorch: {torch.version.cuda}")

torch.cuda.init()
print("After init - CUDA available:", torch.cuda.is_available())



PyTorch version: 2.7.1+cu126
CUDA available: True
CUDA version used by PyTorch: 12.6
After init - CUDA available: True


In [31]:
data_folder = Path("data/title_and_first_sen")
dataset = get_dataset(str(data_folder / "test_data"))
print(dataset)


(Dataset({
    features: ['task_input_a', 'task_input_b', 'task_answer_a', 'task_answer_b'],
    num_rows: 2613
}), TaskDescription(prompt_a='Which subreddit does this belong to? -----', prompt_b='Continue the story. -----', tag_a='reddit', tag_b='story'))


In [2]:
pipeline = pipeline(
    task="text-generation",
    model="huggyllama/llama-7b",
    torch_dtype=torch.float16,
    device=0
)
pipeline("Plants create energy through a process known as")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
Device set to use cuda:0


[{'generated_text': 'Plants create energy through a process known as photosynthesis. Plants convert sunlight and carbon dioxide into a sugar known as glucose. The glucose is then used to create the proteins, fats and carbohydrates that are needed for the growth of the plant.\nCarbon dioxide is one of the main ingredients in the photosynthesis process. The carbon dioxide is obtained from the atmosphere and water molecules are combined with this to produce oxygen. The oxygen is released into the atmosphere as a byproduct of the process and can be used by animals such as humans.\nThe process of photosynthesis also creates an energy gradient between the leaves and the inside of the plant. This gradient allows the plants to act as a primary source of energy for animals that consume them.\nWhat Are the Functions of Photosynthesis?\nHow Does Plants Make Food?'}]